## **Classez des images**

### partie 2/4 : modèle perso

<br>

> #### notebook de mise en oeuvre de création et d’entraînement du modèle personnel, des simulations des différentes valeurs des hyperparamètres et de data augmentation.. <br><br>

<br>


## 0 Imports


### 0.1 Librairies, réglages


In [ ]:
# paths, folders/files
import os, sys, random, re
from os import listdir
from glob import glob
from zipfile import ZipFile
import time

# math, dataframes
import numpy as np
import pandas as pd
from pandarallel import pandarallel
from collections import Counter

# Visualisation
from pprint import pprint
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
import plotly.express as px
# from wordcloud import WordCloud
# from PIL import Image
from optuna.visualization import plot_optimization_history

# Feature engineering
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import manifold, decomposition
from sklearn import cluster, metrics
from sklearn.model_selection import train_test_split
# from sklearn.feature_extraction.text import CountVectorizer

# NN
import tensorflow as tf
from tensorflow.keras.metrics import Accuracy, Precision, Recall, AUC
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

import keras_tuner as kt
import optuna
%pip install optuna-integration
from optuna_integration import TFKerasPruningCallback

# tracking
from mlflow import MlflowClient
import mlflow.keras

# ! REQUIRES CONSOLE COMMAND : mlflow ui
# depuis dossier notebooks
os.environ['MLFLOW_TRACKING_URI'] = './'
# Utilisable seulement en local ?
mlflow.set_tracking_uri("http://localhost:5000")
client = MlflowClient(tracking_uri="http://127.0.0.1:5000")

import pickle

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, GlobalAveragePooling1D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Rescaling, RandomFlip, RandomRotation, RandomZoom
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical


print('\nPython version ' + sys.version)
print('Tensorflow version ' + tf.__version__)
print('Mlflow Autolog compatible with: 2.2.4 <= keras <= 2.6.0')
print('Keras version ' + tf.keras.__version__)
# no autolog... ?

# plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)

# Modify if necessary
num_cores = os.cpu_count()
print(f"\nNumber of CPU cores: {num_cores}")
pandarallel.initialize(progress_bar=False, nb_workers=6)


### 0.2 Variables globales


In [ ]:
nb_classes = 10      # min 2, max 120 (theorique, en pratique : 3, 4 ,5 , 10).
                    # possible creer + en utilisant le ntbk 1

size_wh = 128
target_size=(size_wh, size_wh) # pour grille 5x5, stride (2,2) ?

test_size = 0.15 # same for validation

alea = 42 # pour fixer les ttsplits et tjs travailler sur les mm datasets
# En revanche l'initialisation des poids des modèles restera aléatoire,
# pour pouvoir comparer les resultats sur +ieurs runs.

epochs = 15

nb_runs = 20 # 2-3 le matin, 50 le soir

pickle_path_model = f'./pickle/best_{nb_classes}/ze_model.pkl'
pickle_path_accuracy = f'./pickle/best_{nb_classes}/best_accuracy_val.pkl'
pickle_path_params = f'./pickle/best_{nb_classes}/best_params.pkl'

try:
    best_acc = pickle.load(open(pickle_path_accuracy, 'rb'))
    best_parameters = pickle.load(open(pickle_path_params, 'rb'))
except:
    best_acc = 0.3

print(best_acc, '\n')
pprint(best_parameters)


### 0.3 Fonctions


In [ ]:
# tracking manuel

model_results = []

def affichage_results(multiple=True, best_acc=best_acc):
    """Tracking manuel de nos modèles (data, params, tps, scores) pour comparaison."""

    print(f'{nb_classes} classes prédites')
    print(f'dim images : {size_wh} x {size_wh} x 3')
    print(f'size validation & testing sets : {test_size}')
    print(f'nb epochs : {epochs}', '\n')
    print(f'current best accuracy (validation set) : {best_acc}')

    if multiple:
        print(f'nb runs: {nb_runs}', '\n')

    # Create a DataFrame from the list of model results
    model_comparison_df = pd.concat([pd.DataFrame(model_results)], ignore_index=True)

    # Sort the DataFrame by precision in descending order (higher is better)
    model_comparison_df.sort_values(by='accuracy_val_moy', ascending=False, inplace=True)

    # Display the sorted DataFrame
    display(model_comparison_df)


# tracking mlflow

def create_mlflow_experiment(
    experiment_name: str, artifact_location: str, tags: dict[str, str]
) -> str:
    """
    Create a new mlflow experiment with the given name and artifact location.

    Parameters:
    ----------
    experiment_name: str
        The name of the experiment to create.
    artifact_location: str
        The artifact location of the experiment to create.
    tags: dict[str,Any]
        The tags of the experiment to create.

    Returns:
    -------
    experiment_id: str
        The id of the created experiment.
    """
    try:
        experiment_id = mlflow.create_experiment(
            name=experiment_name, artifact_location=artifact_location, tags=tags
        )
    except:
        print(f"Experiment {experiment_name} already exists.")
        experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id

    mlflow.set_experiment(experiment_name=experiment_name)

    return experiment_id


def get_mlflow_experiment(
    experiment_id: str = None, experiment_name: str = None
) -> mlflow.entities.Experiment:
    """
    Retrieve the mlflow experiment with the given id or name.

    Parameters:
    ----------
    experiment_id: str
        The id of the experiment to retrieve.
    experiment_name: str
        The name of the experiment to retrieve.

    Returns:
    -------
    experiment: mlflow.entities.Experiment
        The mlflow experiment with the given id or name.
    """
    if experiment_id is not None:
        experiment = mlflow.get_experiment(experiment_id)
    elif experiment_name is not None:
        experiment = mlflow.get_experiment_by_name(experiment_name)
    else:
        raise ValueError("Either experiment_id or experiment_name must be provided.")

    return experiment


### 0.4 Metriques


In [ ]:
# J'utiliserai tjs les noms anglais des métriques ici,
# pour éviter la confusion précision (fr) != precision (en),
# et pour simplement garder les noms des fonctions importées depuis tf.keras.metrics

# Nous sommes dans un cas de classification "classique", 1 classe prédite.
# Une première métrique simple et intuitive est donc l'accuracy :
# nb de prédictions correctes / nb total de prédictions.
# Cette métrique nous suffit déjà pour comparer et optimiser nos modèles.

# Si l'on souhaite étudier + en détail les prédictions des modèles, on utilisera
# la precison et le recall

# Precison (TP / (TP + FP)) :
# Une précision élevée signifie que si une classe est prédite par le modèle,
# alors il y a une forte probabilité (égale à la precision)
# que le chien appartienne en effet à cette classe.

# Recall (TP / (TP + FN)) :
# pour évaluer la capacité des modèles à identifier toutes les instances positives.
# Exemple : Si notre precision est égale à 1, c'est parfais, cela signifie que
# pour une une classe au moins, toutes les valeurs prédites par le modèle sont correctes.
# Cependant, il est possible que cela ne concerne que très peu de cas (mettons, 1 ou 2 prédictions)
# et qu'à côté de cela le modèle a pu faire des milliers de prédictions incorrectes,
# la precision seule ne nous le dit pas.

# Comme l'amélioration de la precision se fait svt au détriment du recall, en pratique
# on combine souvent les 2 avec le f1score (= moyenne harmonique)
# (2 x precision x recall) / (precision + recall)
# qui nous donne directement une idée du compromis precision / recall

# petit souci : le f1score et keras, c'est tout une histoire...
# Dans les versions récentes (depuis la 2.15.0 il me semble), le f1score est directement intégré
# au module metrics. Le problème est que conda n'arrive pas à résoudre un env avec ces versions,
# incompatibles avec les requirements d'autres packages dans l'env.

# Dans les versions + anciennes de keras, le f1score était dans un autre module, "addons",
# mais ce moule est désormais déprécié.
# Solutions : définir un f1score custom
# Pas évident car tensorflow est très pointilleux sur les types d'objets qu'il accepte.
# Exemple : cette fonction est construite en utilisant 2 fonctions tf.keras,
# pourtant elle renvoit une erreur si on tente de l'utiliser parce que le type retourné est
# un float.

def f1score(y_true, y_pred):
    precision = Precision(y_true, y_pred)
    recall = Recall(y_true, y_pred)
    f1 = (2*precision*recall) / (precision + recall)

    return f1

# Autre solution, utiliser le f1score de sk-learn. Compatibilité logging mlflow ?

metrics=[
    'Accuracy',
    # f1score,
    # AUC(),
]


### 0.4 Data


In [ ]:
data = pd.read_csv(f'./data/data_{nb_classes}_classes.csv', sep=',')

print(data.shape)
data.head()


### 0.5 Etude de faisabilité (sort of)


In [ ]:
# Ici l'étude de faisabilité préconisée par la méthode Agile n'est pas vraiment utile en tant que telle
# (On sait que le projet est faisable). Il s'agit plutôt de pouvoir observer le travail effectué par le
# bloc d'encodage, sans utiliser d'algorithme de prediction supervisée
# (algo classique ou plutôt, ici, bloc des layers fully connected)

images_features = []

for image_file in data["photo_path"] :
    image = load_img(image_file, target_size=(180, 180))
    image = img_to_array(image)
    images_features.append(image)

images_features = np.asarray(images_features)
images_features.shape


In [ ]:
# Reshape images to flatten them into vectors
flattened_images = images_features.reshape(images_features.shape[0], -1)
print(flattened_images.shape)

# Normalize the data
scaler = StandardScaler()
normalized_images = scaler.fit_transform(flattened_images)


### 0.6 Réduction dim


In [ ]:
# PCA

print(normalized_images.shape) # same as flattened_images

pca = decomposition.PCA(n_components=0.99)
feat_pca= pca.fit_transform(normalized_images)

print(feat_pca.shape)

# dimention divisée par 200 (presque), en conservant 99% de la variance !


In [ ]:
# Plot explained variance ratio
plt.figure(figsize=(8, 6))
plt.plot(range(1, pca.n_components_ + 1), pca.explained_variance_ratio_.cumsum(), marker='o', linestyle='--', color='#3af')
plt.xlabel('Number of Principal Components')
plt.ylabel('Cumulative Explained Variance Ratio')
plt.title('Cumulative Explained Variance Ratio vs. Number of Principal Components')
plt.grid(True)
plt.show()

# Pourquoi on a besoin du tsne pour la visu : en 2D ou même en 3D, les 3 premiers vecteurs propres
# # fournis par la PCA ne captent "que" (environ) un tiers de l'information.
# Ce qu'on verrait serait très déformé par les projections successives de la PCA.
# tester ?


### 0.7 tsne


In [ ]:
# t-sne

tsne = manifold.TSNE(n_components=2, perplexity=30, n_iter=2000, init='random', random_state=6)
X_tsne = tsne.fit_transform(feat_pca)


In [ ]:
# encodage target

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit(data["breed"])

data["target"] = label_encoder.transform(data["breed"])

display(data.head(1))
data.tail(1)


In [ ]:
df_tsne = pd.DataFrame(X_tsne, columns=['tsne1', 'tsne2'])
df_tsne["class"] = data["target"]

plt.figure(figsize=(8,5))
sns.scatterplot(
    x="tsne1", y="tsne2",
    hue="class",
    palette=sns.color_palette('tab10', n_colors=3), s=50, alpha=0.6,
    data=df_tsne,
    legend="brief")

plt.title('TSNE selon les vraies classes', fontsize = 30, pad = 35, fontweight = 'bold')
plt.xlabel('tsne1', fontsize = 26, fontweight = 'bold')
plt.ylabel('tsne2', fontsize = 26, fontweight = 'bold')
plt.legend(prop={'size': 14})

plt.show()

# Ca marche moins bien sans extraction de features !
# On retentera en fin de notebook, en utilisant notre modèle.


### 1 Création d'un premier modèle


In [ ]:
# Notre objectifs principal ici est
# de pouvoir observer / comprendre la fonction des différentes layers utilisées.

# Pour cela, nous allons commencer par une architecture très simple :
# le but n'est pas d'avoir le modèle le + performant possible.
# (irréaliste ici car on n'aurait ni le tps ni les ressources pour l'entrainer)
# (en revanche, voir le notebook 3, transfer learning, pour une comparaison de modèles + complexes)

# Première idée :
# Notre modèle de base sera donc inspiré d'AlexNet, dont l'architecture est :

# "AlexNet contains eight layers: the first five are convolutional layers,
# some of them followed by max-pooling layers, and the last three are fully connected layers.
# [...] The entire structure can be written as:

# (CNN -> RN -> MP)^2 -> (CNN^3 -> MP) -> (FC -> DO)^2 -> Linear -> softmax

# where
# CNN = convolutional layer (with ReLU activation)
# RN = local response normalization
# MP = maxpooling
# FC = fully connected layer (with ReLU activation)
# Linear = fully connected layer (without activation)
# DO = dropout

# It used the non-saturating ReLU activation function, which showed improved training performance
# over tanh and sigmoid." (wiki)


### 1.1 LeNet inspired architecture


In [ ]:
# Problème : 8 groupes de layers... (16 individuelles, en fait) C'est déjà beaucoup !
# On peut faire + simple, au moins pour commencer.

# Voyons de quoi sera capable un modèle inspiré plutôt par LeNet-5
# et par ce notebook : https://www.kaggle.com/code/schmoyote/simple-cnn-architecture-for-image-classification/notebook


def model_v1():
    model = Sequential()
    model.add(Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=(size_wh, size_wh, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(16, kernel_size=(5, 5), activation='tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    # model.add(Dense(120, activation='tanh'))
    model.add(Dense(60, activation='tanh'))
    model.add(Dense(nb_classes, activation='softmax'))

    return model


model = model_v1()

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

model.summary()


### 1.2 feature engineering


In [ ]:
feature = "photo_path"


def feature_to_array(feature="photo_path", norm=False, shape_result=False):
    X_feature = []

    for image_file in data[feature] :
        image = load_img(image_file, target_size=target_size)
        image = img_to_array(image)
        if norm:
            image = image / 255
        X_feature.append(image)

    X_feature = np.asarray(X_feature)

    if shape_result:
        pprint(X_feature[:1])
        print('\n', "Shape of X_train:", X_feature.shape)
        # ok

    return X_feature


X_feature = feature_to_array(shape_result=True)


### 1.3 label encoding target


In [ ]:
y_target = np.asarray(data["target"])
print(y_target.shape)
pprint(y_target)


### 1.4 train test split


In [ ]:
X_train_val, X_test, y_train_val, y_test = train_test_split(X_feature, y_target, test_size=test_size,
                                                            shuffle=True, random_state=alea,
                                                            stratify=y_target) # important

X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=test_size,
                                                            shuffle=True, random_state=alea,
                                                            stratify=y_train_val)

print(X_train.shape)
print(X_val.shape)
print(X_test.shape, '\n')

print(y_train.shape)
print(y_val.shape)
print(y_test.shape, '\n')


### 1.5 one hot encoding (targets)


In [ ]:
# One-hot encode target values after the split to avoid data leakage

y_train_ohe = tf.keras.utils.to_categorical(y_train)
y_val_ohe = tf.keras.utils.to_categorical(y_val)
y_test_ohe = tf.keras.utils.to_categorical(y_test)


### 1.6 Training


In [ ]:
# Train the model
model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
          validation_data=(X_val, y_val_ohe), verbose=0) # verbose=0 avoids crashing cell later


### 1.7 Evaluation


In [ ]:
# On overfit dès le début ??

# Evaluate the model
val_loss_ref, val_acc_ref = model.evaluate(X_val, y_val_ohe)
print(f'Val accuracy: {val_acc_ref}', '\n')

# En prédisant au hasard on aurait une chance sur 3, autrement dit
# ce modèle fait des prédictions quasi-aléatoires.
# Pas terrible, mais c'est un début !

# avant, tester sur photos d'origine (pour évaluer l'utilité du prétraitement effectué)



### save


In [ ]:
parameters = {
    "model": "V1",
    "preprocessing": 'no_norm',
    "feature": feature,
    'target': f'{nb_classes}_classes',
    'dim images': f'{size_wh} x {size_wh} x 3',
    'size val test sets': test_size,
    'nb epochs': epochs
}


def save_model_if_better_than_the_best(model=model, acc=val_acc_ref, best_params=parameters):

    global best_acc

    if acc > best_acc:
        print(f'Improving! Accuracy up to {acc}')
        best_acc = acc
        # erreur logging, saving instead
        # using mlflow
        # mlflow.keras.save_model(model, "./best_model") # works only once
        # using pickle
        with open(pickle_path_model, 'wb') as f:
            pickle.dump(model, f)
        # also save best accuracy
        with open(pickle_path_accuracy, 'wb') as f:
            pickle.dump(best_acc, f)
        # and corresponding best params
        with open(pickle_path_params, 'wb') as f:
            pickle.dump(best_params, f)


save_model_if_better_than_the_best()


In [ ]:
# petit tracking manuel
results = {'model': 'V1',
            'df': 'data_3_classes',
            'feature': feature,
            'accuracy_val_moy': val_acc_ref,
            'time_fit': 'to do',
            'time_predict':'to do'
            }

# Append a new row for this model
model_results.append(results)


### 1.8 test utilité prétraitements


In [ ]:
experiment_id = create_mlflow_experiment(
    experiment_name="analyse_pretraitements",
    artifact_location="./mlruns/artifacts",
    tags={"model": "V1", "preprocessing": 'no_norm', "features": "7_processed_features", 'target': '3_classes'},
)


In [ ]:
# resized


def test_feature(df=data, feature='photo_path', epochs=epochs):
    parameters = {
        "model": "V1",
        "preprocessing": 'no_norm',
        "feature": feature,
        'target': f'{nb_classes}_classes',
        'dim images': f'{size_wh} x {size_wh} x 3',
        'size val test sets': test_size,
        'nb epochs': epochs
    }
    mlflow.log_params(parameters)

    # feature, target
    X_feature = feature_to_array(feature)
    y_target = np.asarray(df["target"])

    #tt split
    X_train_val, X_test, y_train_val, y_test = train_test_split(X_feature, y_target, test_size=0.1,
                                                                shuffle=True, random_state=alea,
                                                                stratify=y_target) # important

    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1,
                                                                shuffle=True, random_state=alea,
                                                                stratify=y_train_val)

    # ohe cible
    y_train_ohe = tf.keras.utils.to_categorical(y_train)
    y_val_ohe = tf.keras.utils.to_categorical(y_val)
    y_test_ohe = tf.keras.utils.to_categorical(y_test)

    # model
    model = model_v1()
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

    model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
            validation_data=(X_val, y_val_ohe), verbose=0)

    _, val_acc = model.evaluate(X_val, y_val_ohe, verbose=0)

    save_model_if_better_than_the_best(model=model, acc=val_acc, best_params=parameters)

    metrix = {
        "accuracy_val": val_acc
    }
    # multiple metrics
    mlflow.log_metrics(metrix)

    print(f'Val accuracy (feature={feature}): {val_acc}')

    results = {'model': 'V1',
            'df': 'data_3_classes',
            'feature': feature,
            'accuracy_val_moy': val_acc,
            'time_fit': 'to do',
            'time_predict':'to do'
            }

    # Append a new row for this model
    model_results.append(results)

    return val_acc


# _ = test_feature(df=data, feature='resized')


# 0.6 de precision sans pretraitement (parfois 0.3 ??), 0.3 avec.
# Notre prétraitement semble (très) contre-productif. Ajuster dim, filtres ? (trop flou ?)
# Comme prétraitement, les méthodes .preprocessing() de keras consistent svt seulement en
# redimensionnemt + normalisation.
# faire pareil ?


In [ ]:
# nested

features_to_test = ['resized', 'expo', 'contraste', 'denoised_g', 'denoised_bi', 'denoised_nl',
                    'preprocess_complet']


experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))

with mlflow.start_run(run_name="7_processed_features_once", experiment_id=experiment_id) as parent:
    all_metrics = {}

    for feature in features_to_test:
        print('\n', feature)
        with mlflow.start_run(run_name=f'{feature}', nested=True) as child:
            print(f'RUN ID child_{feature}: {child.info.run_id}', '\n')
            accuracy_val = test_feature(df=data, feature=feature)
            all_metrics[f'accuracy_{feature}'] = accuracy_val

    print('\n')
    pprint(all_metrics)
    mlflow.log_metrics(all_metrics)

# results
# 'resized' 0.32 ???
# 'expo' 0.63
# 'contraste' 0.32

# rappel
print(f'\nRappel : Val accuracy (photo_path original): {val_acc_ref}')


### Comparaison


In [ ]:
affichage_results(multiple=False, best_acc=best_acc)

# On y voit déjà (un peu) + clair :
# Chaque étape de notre prétraitement semble + ou - détériorer la qualité des prédictions.

# Encore que... ?
# ??? resized et photo_path devraient donner des résultats bien + proches, non ??
# sets identiques sauf dim, et redim lors de création de X_feature
# ... devraient être exactement identiques, et donc avoir des resultats proches

# En fait d'un run à l'autre, les prédictions varient énormément...
# difficile du coup d'évaluer l'impact de nos prétraitements.
# moyenne sur +ieurs runs ?

model_results = []

# (Très) svt le modèle ne parvient pas à "train ses layers", et l'accuracy des prédictions
# sur le jeu de validation reste au niveau de prédictions random.


### Mean multiple runs


In [ ]:
experiment_id = create_mlflow_experiment(
    experiment_name='analyse_pretraitmt_multiple_runs',
    artifact_location="./mlruns/artifacts",
    tags={"model": "V1", "preprocessing": 'no_norm', "features": "all_8_features", 'target': '3_classes'},
)


In [ ]:
#


def test_feature_n_times(feature='photo_path', epochs=epochs, n=nb_runs):
    """

    """
    parameters = {
        "model": "V1",
        "preprocessing": 'no_norm',
        "feature": feature,
        'target': f'{nb_classes}_classes',
        'dim images': f'{size_wh} x {size_wh} x 3',
        'size val test sets': test_size,
        'nb epochs': epochs
    }
    mlflow.log_params(parameters) # child

    results_val_acc, results_time_fit,  results_time_predict = [], [], []

    for i in range(n):

        with mlflow.start_run(run_name=f'run_{i}', nested=True) as grand_kid:
            print(f'\nRun {i}\n')
            print("RUN ID ", grand_kid.info.run_id )
            mlflow.log_params(parameters) # grand_kid

            # feature, target
            X_feature = feature_to_array(feature)
            y_target = np.asarray(data["target"])

            X_train_val, X_test, y_train_val, y_test = train_test_split(X_feature, y_target, test_size=0.1,
                                                                        shuffle=True, random_state=alea,
                                                                        stratify=y_target) # important

            X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1,
                                                                        shuffle=True, random_state=alea,
                                                                        stratify=y_train_val)

            y_train_ohe = tf.keras.utils.to_categorical(y_train)
            y_val_ohe = tf.keras.utils.to_categorical(y_val)
            y_test_ohe = tf.keras.utils.to_categorical(y_test)

            # model
            model = model_v1()

            # Compile the model
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

            # fit model and time it
            time_fit_start = time.time() # returns the time in seconds since the epoch.
            model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
                    validation_data=(X_val, y_val_ohe), verbose=0)
            time_fit_end = time.time()
            time_fit = time_fit_end - time_fit_start

            # time predictions
            time_predict_start = time.time()
            _, val_acc = model.evaluate(X_val, y_val_ohe) # , verbose=0 ?
            time_predict_end = time.time()
            time_predict = time_predict_end - time_predict_start

            # print(f'Val accuracy (feature={feature}): {val_acc}')

            save_model_if_better_than_the_best(model=model, acc=val_acc, best_params=parameters)

            metriq = {
                "accuracy_val": val_acc,
                "time_fit": time_fit,
                "time_predict": time_predict
            }
            mlflow.log_metrics(metriq)

            results_val_acc.append(val_acc)
            results_time_fit.append(time_fit)
            results_time_predict.append(time_predict)

            # Log the model doesn't work ??
            # mlflow.keras.log_model(model=model, artifact_path="good_model",registered_model_name='best_model')

    # moyennes
    mean_val_acc = np.mean(results_val_acc)
    mean_time_fit = np.mean(results_time_fit)
    mean_time_predict = np.mean(results_time_predict)

    # écarts-types (utile ici pour le score, afin d'avoir une idée de la "régularité" des résultats)
    # les tps d'entrainement / prédiction st bcp + stables
    std_val_acc = np.std(results_val_acc)

    metrix = {
        "accuracy_val_moy": mean_val_acc,
        "accuracy_val_std": std_val_acc,
        "time_fit_moy": mean_time_fit,
        "time_predict_moy": mean_time_predict
    }
    for run in range(nb_runs):
        metrix[f'accuracy_val_run_{run}'] = results_val_acc[run]

    # multiple metrics
    mlflow.log_metrics(metrix)

    results = {'model': 'V1',
        'df': 'data_3_classes',
        'feature': feature,
        'accuracy_val_moy': mean_val_acc,
        'accuracy_val_std': std_val_acc,
        'time_fit_moy (s)': mean_time_fit,
        'time_predict_moy (s)': mean_time_predict,
        }

    # Append a new row for this model
    model_results.append(results)

    print('\n', f'Accuracy val {feature}, moyenne : {mean_val_acc}')
    print(f'Accuracy val {feature}, écart-type : {std_val_acc}')

    # + tard
    # log model
    # mlflow.sklearn.log_model(sk_model=knn_ref, artifact_path="special_knn")

     # Save a plot
    # plt.savefig("./artifacts/jaccard_plot_3_4_tags.png")
    # Log the saved figure using MLflow
    # mlflow.log_artifact("./artifacts/jaccard_plot_3_4_tags.png")

    return metrix


# _ = test_feature_n_times()

#


In [ ]:
# nested

all_features_to_test = ['photo_path'] + features_to_test


experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))


with mlflow.start_run(run_name="test_all_8_features", experiment_id=experiment_id) as parent:
    parameters = {
        "model": "V1",
        "preprocessing": 'no_norm',
        "feature": feature,
        'target': f'{nb_classes}_classes',
        'dim images': f'{size_wh} x {size_wh} x 3',
        'size val test sets': test_size,
        'nb epochs': epochs
    }
    mlflow.log_params(parameters) # parent

    all_metrics = {}

    for feature in all_features_to_test:
        print('\n', feature)
        with mlflow.start_run(run_name=f'{feature}', nested=True) as child:
            print(f'RUN ID child_{feature}: {child.info.run_id}', '\n')
            metrix = test_feature_n_times(feature=feature)
            all_metrics[f'accuracy_val_moy_{feature}'] = metrix['accuracy_val_moy']

    print('\n')
    pprint(all_metrics)
    mlflow.log_metrics(all_metrics) # perfect


# results
# 'resized' 0.32 ???
# 'expo' 0.63
# 'contraste' 0.32


In [ ]:
affichage_results(best_acc=best_acc)

# Enfin des résultats interprétables !
# Cpdt le nb de runs est encore très faible, et les résultats st très proches.

# model_results = []

# log / picle model. size ?

# les pretraitements expo / contraste semblent bénéfiques,
# en revanche le filtre gaussien mm léger ici semble avoir un impact légèrement négatif
# sur les prédictions du modèle. Le modèle n'aime pas le flou !

# (encore une fois, ces "conclusions" sont à prendre "avec des pincettes", en attendant
# des runs supplémentaires.)


In [ ]:
# A propos de la fonction d'activation tanh

# Choisi tanh pour commencer, une fonction d'activation "historique",
# +tôt que relu (+ utilisée dans les archis des modèles + récents)
# parce que la tangeante hyperbolique agit comme un scaler :
# Elle centre les valeurs autour de 0, sur l'intervalle [-1, 1]

# du coup pas besoin de normaliser nos pixels au préalable.
# Quand on testera relu, il faudra comparer avec ou sans normalisation.

# Autre caractéristique intéressante de la fonction tanh :

# La symétrie par rapport à l'origine + normalisation peuvent aider les algorithmes à optimiser le modèle,
# en empêchant les gradients de devenir trop proches de 0 (problème fréquent du 'vanishing gradient')
# ou au contraire trop importants ('exploding gradient') pdt la backpropagation.
# Il est cpdt tjs possible de rencontrer un vanishing gradient.


## 2 Ameliorations


In [ ]:
# 3 pistes -> améliorations possibles :

# retour au preprocessing,
# data augmentation,
# améliorer le model lui-même.

# Nous allons commencer par le modèle, car parvenir à obtenir un modèle + consistent
# (= faire baisser la sdt) rendrait l'analyse des pretraitements et de la data augmentation
# bcp + fiable.


### keras-tuner, archi 1


In [ ]:
experiment_id = create_mlflow_experiment(
    experiment_name='keras_tuner_1_archi',
    artifact_location="./mlruns/artifacts",
    tags={"model": "V2", "preprocessing": 'no_norm', "feature": "photo_path", 'target': '3_classes'},
)

experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))


In [ ]:
# tester Optuna ? (compatible sk-learn + tracking ui)
# kt très facile à intégrer ici, puisque nous utilisons uniquement keras
# syntaxe légèrement + simple (mais les 2 st assez proches)
# (sauf / f1score ?)

# Commençons par qq hyperparams de l'architecture générale de notre modèle :
# (On espère "alléger" le modèle, pour le rendre plus facile a entrainer.)
# nb de filtres (pour les 2 couches de convolution)
# nb neurons (first fully-connected layer)


def build_model(hp):

    model = Sequential()
    model.add(Conv2D(hp.Int("conv_1_units", min_value=4, max_value=32, step=4),
                     kernel_size=(5, 5), activation='tanh', input_shape=(size_wh, size_wh, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(hp.Int("conv_2_units", min_value=4, max_value=32, step=4),
                     kernel_size=(5, 5), activation='tanh'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(hp.Int("dense_units", min_value=32, max_value=192, step=32), activation='tanh'))
    model.add(Dense(nb_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


def test_archi_n_times(feature='photo_path', epochs=epochs, n=nb_runs, model_name='V2_kt',
                       norm=False, dir='keras_tuner/archi_1'):
    """
    """

    tuner = kt.Hyperband(build_model,
                         objective='val_accuracy',
                         max_epochs=epochs,
                         factor=3,
                         directory=dir,
                         project_name=model_name)

    results_val_acc, results_time_fit,  results_time_predict = [], [], []

    for i in range(n):
        with mlflow.start_run(run_name=f'run_{i}', nested=True) as kid:
            print(f'\nRun {i}\n')
            print("RUN ID ", kid.info.run_id)

            # feature, target
            X_feature = feature_to_array(feature, norm=norm)
            y_target = np.asarray(data["target"])

            X_train_val, X_test, y_train_val, y_test = train_test_split(X_feature, y_target, test_size=0.1,
                                                                        shuffle=True, random_state=alea,
                                                                        stratify=y_target) # important

            X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1,
                                                                        shuffle=True, random_state=alea,
                                                                        stratify=y_train_val)

            y_train_ohe = tf.keras.utils.to_categorical(y_train)
            y_val_ohe = tf.keras.utils.to_categorical(y_val)
            y_test_ohe = tf.keras.utils.to_categorical(y_test)

            tuner.search(X_train, y_train_ohe, validation_data=(X_val, y_val_ohe))

            # Get the optimal hyperparameters
            best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

            # Build the model with the optimal hyperparameters
            model = tuner.hypermodel.build(best_hps)

            # Fit model and time it
            time_fit_start = time.time()
            model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
                    validation_data=(X_val, y_val_ohe), verbose=0)
            time_fit_end = time.time()
            time_fit = time_fit_end - time_fit_start

            # Time predictions
            time_predict_start = time.time()
            _, val_acc = model.evaluate(X_val, y_val_ohe) # , verbose=0 ?
            time_predict_end = time.time()
            time_predict = time_predict_end - time_predict_start

            # print(f'Val accuracy (feature={feature}): {val_acc}', '\n')

            parameters.update(best_hps.values)
            mlflow.log_params(parameters)

            save_model_if_better_than_the_best(model=model, acc=val_acc, best_params=parameters)

            metriq = {
                "accuracy_val": val_acc,
                "time_fit": time_fit,
                "time_predict": time_predict
            }
            mlflow.log_metrics(metriq)


            results_val_acc.append(val_acc)
            results_time_fit.append(time_fit)
            results_time_predict.append(time_predict)

    # Calculate averages
    mean_val_acc = np.mean(results_val_acc)
    mean_time_fit = np.mean(results_time_fit)
    mean_time_predict = np.mean(results_time_predict)

    # Calculate standard deviations
    std_val_acc = np.std(results_val_acc)

    metrix = {
        "accuracy_val_moy": mean_val_acc,
        "accuracy_val_std": std_val_acc,
        "time_fit_moy": mean_time_fit,
        "time_predict_moy": mean_time_predict
    }
    for run in range(nb_runs):
        metrix[f'accuracy_val_run_{run}'] = results_val_acc[run]

    # multiple metrics
    mlflow.log_metrics(metrix)

    results = {'model': model_name,
               'df': 'data_3_classes',
               'feature': feature,
               'accuracy_val_moy': mean_val_acc,
               'accuracy_val_std': std_val_acc,
               'time_fit_moy (s)': mean_time_fit,
               'time_predict_moy (s)': mean_time_predict,
              }

    # Append a new row for this model
    model_results.append(results)

    print(f'Accuracy val {feature}, moyenne : {mean_val_acc}')
    print(f'Accuracy val {feature}, écart-type : {std_val_acc}', '\n')

    print("Best Hyperparameters:")
    print(best_hps.values)

    return metrix


#  _ = test_archi_n_times()

# wow
# Reloading Tuner from keras_tuner_dir/test_kt/tuner0.json
# kt ne perd pas de tps et récupère directement les hyperparamètres
# Au final il semble légèrement + rapide d'utiliser kt !


In [ ]:
with mlflow.start_run(run_name="test_archi_1", experiment_id=experiment_id) as parent:
    parameters = {
        "model": "V2",
        "preprocessing": 'no_norm',
        "feature": 'originale, photo_path',
        'target': f'{nb_classes}_classes',
        'dim images': f'{size_wh} x {size_wh} x 3',
        'size val test sets': test_size,
        'nb epochs': epochs
    }
    mlflow.log_params(parameters)

    all_metrics = {}

    metrix = test_archi_n_times()

    all_metrics[f'accuracy_val_moy_{feature}'] = metrix['accuracy_val_moy']

    print('\n')
    pprint(all_metrics)
    mlflow.log_metrics(all_metrics)


In [ ]:
affichage_results(best_acc=best_acc)

# Enfin un écart significatif !
# std tjs élevé (tjs l'aléatoire / initialisation)


### keras-tuner, fonction d'activation


In [ ]:
experiment_id = create_mlflow_experiment(
    experiment_name='keras_tuner_2_activation',
    artifact_location="./mlruns/artifacts",
    tags={"model": "V3", "preprocessing": 'no_norm', "feature": "photo_path", 'target': '3_classes'},
)

experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))


In [ ]:
# Continuons en testant la fonction d'activation relu

# tester Optuna ? (compatible sk-learn)
# kt très facile à intégrer ici, puisque nous utilisons uniquement keras
# (sauf / f1score ?)

# Commençons par qq hyperparams de l'architecture générale de notre modèle :
# nb de filtres (pour les 2 couches de convolution)
# nb neurons (first fully-connected layer)

def build_model(hp):

    model = Sequential()
    model.add(Conv2D(hp.Int("conv_1_units", min_value=4, max_value=32, step=4),kernel_size=(5, 5),
                     activation=hp.Choice("activation", ["relu", "tanh"]),
                     input_shape=(size_wh, size_wh, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(hp.Int("conv_2_units", min_value=4, max_value=32, step=4),
                     kernel_size=(5, 5), activation=hp.Choice("activation", ["relu", "tanh"])))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    model.add(Dense(hp.Int("dense_units", min_value=32, max_value=192, step=32),
                    activation=hp.Choice("activation", ["relu", "tanh"])))
    model.add(Dense(nb_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


with mlflow.start_run(run_name="test_activation", experiment_id=experiment_id) as parent:
    parameters = {
        "model": "V3",
        "preprocessing": 'no_norm',
        "feature": 'originale, photo_path',
        'target': f'{nb_classes}_classes',
        'dim images': f'{size_wh} x {size_wh} x 3',
        'size val test sets': test_size,
        'nb epochs': epochs
    }
    mlflow.log_params(parameters)

    all_metrics = {}

    metrix = test_archi_n_times(model_name='V3_kt_activ', dir='keras_tuner/activ')

    all_metrics[f'accuracy_val_moy_{feature}'] = metrix['accuracy_val_moy']

    print('\n')
    pprint(all_metrics)
    mlflow.log_metrics(all_metrics)


# {'conv_1_units': 28, 'activation': 'tanh', 'conv_2_units': 4, 'dense_units': 96


In [ ]:
affichage_results(best_acc=best_acc)

# tjs très aléatoire.
# std ?

# relu sans normalisation ici


### normalisation + fonction d'activation


In [ ]:
experiment_id = create_mlflow_experiment(
    experiment_name='keras_tuner_2_activation_norm',
    artifact_location="./mlruns/artifacts",
    tags={"model": "V3", "preprocessing": 'normalisation', "feature": "photo_path", 'target': '3_classes'},
)

experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))


In [ ]:
with mlflow.start_run(run_name="activation_norm", experiment_id=experiment_id) as parent:
    parameters = {
        "model": "V3",
        "preprocessing": 'norm',
        "feature": 'originale, photo_path',
        'target': f'{nb_classes}_classes',
        'dim images': f'{size_wh} x {size_wh} x 3',
        'size val test sets': test_size,
        'nb epochs': epochs
    }
    mlflow.log_params(parameters)

    all_metrics = {}

    metrix = test_archi_n_times(model_name='V3_kt_activ_norm', norm=True, dir='keras_tuner/norm_activ')

    all_metrics[f'accuracy_val_moy_{feature}'] = metrix['accuracy_val_moy']

    print('\n')
    pprint(all_metrics)
    mlflow.log_metrics(all_metrics)


In [ ]:
affichage_results(best_acc=best_acc)


### optuna


In [ ]:
experiment_id = create_mlflow_experiment(
    experiment_name='optuna_1_activ',
    artifact_location="./mlruns/artifacts",
    tags={"model": "V4", "preprocessing": 'no_norm', "feature": "photo_path", 'target': '3_classes'},
)

experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))


In [ ]:
# Contrairement à keras tuner, qui sauvegarde par défaut le résultat obtenu sur le disque,
# optuna refait les tests à chaque run


def build_model(conv_1_units, filter1, conv_2_units, filter2, fc_1_units):
    model = Sequential()
    model.add(Conv2D(conv_1_units, kernel_size=(filter1, filter1),
                     activation="tanh",
                     input_shape=(size_wh, size_wh, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(conv_2_units, kernel_size=(filter2, filter2),
                     activation="tanh"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(fc_1_units, activation='tanh'))
    model.add(Dense(nb_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


def objective(trial):
    # Define a pruning callback
    pruning_callback = TFKerasPruningCallback(trial, "val_accuracy")

    # Parameters to optimize
    conv_1_units = trial.suggest_int('conv_1_units', 4, 32, step=4)
    filter1 = trial.suggest_int('filter1', 3, 5, step=2)
    conv_2_units = trial.suggest_int('conv_2_units', 4, 32, step=4)
    filter2 = trial.suggest_int('filter2', 3, 5, step=2)
    fc_1_units = trial.suggest_int('fc_1_units', 10, 100, step=10)

    # Build and compile the model with the suggested parameters
    model = build_model(conv_1_units, filter1, conv_2_units, filter2, fc_1_units)

    # Train the model
    history = model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
                        validation_data=(X_val, y_val_ohe),
                        callbacks=[pruning_callback], verbose=0)

    # Evaluate the model on the validation set
    val_acc = history.history['val_accuracy'][-1]

    return val_acc


def test_optuna_n_times(feature='photo_path', epochs=epochs, n=nb_runs, norm=True, model_name='V4_optuna'):
    results_val_acc, results_time_fit,  results_time_predict = [], [], []

    for i in range(n):
        with mlflow.start_run(run_name=f'run_{i}', nested=True) as kid:
            print(f'\nRun {i}\n')
            print("RUN ID ", kid.info.run_id)

            # feature, target
            X_feature = feature_to_array(feature, norm=norm)
            y_target = np.asarray(data["target"])

            X_train_val, X_test, y_train_val, y_test = train_test_split(X_feature, y_target, test_size=0.1,
                                                                        shuffle=True, random_state=alea,
                                                                        stratify=y_target)

            X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1,
                                                                        shuffle=True, random_state=alea,
                                                                        stratify=y_train_val)

            y_train_ohe = tf.keras.utils.to_categorical(y_train)
            y_val_ohe = tf.keras.utils.to_categorical(y_val)
            y_test_ohe = tf.keras.utils.to_categorical(y_test)

            study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
            study.optimize(objective, n_trials=10)

            # plot_optimization_history(study)

            # Fetch the best parameters
            best_trial = study.best_trial
            best_params = best_trial.params

            # Build the model with the best parameters
            model = build_model(**best_params)

            # Fit model and time it
            time_fit_start = time.time()
            model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
                    validation_data=(X_val, y_val_ohe), verbose=0)
            time_fit_end = time.time()
            time_fit = time_fit_end - time_fit_start

            # Time predictions
            time_predict_start = time.time()
            _, val_acc = model.evaluate(X_val, y_val_ohe)
            time_predict_end = time.time()
            time_predict = time_predict_end - time_predict_start

            # print(f'Val accuracy (feature={feature}): {val_acc}')

            parameters.update(best_params)
            mlflow.log_params(parameters)

            save_model_if_better_than_the_best(model=model, acc=val_acc, best_params=parameters)

            metriq = {
                "accuracy_val": val_acc,
                "time_fit": time_fit,
                "time_predict": time_predict
            }
            mlflow.log_metrics(metriq)

            results_val_acc.append(val_acc)
            results_time_fit.append(time_fit)
            results_time_predict.append(time_predict)

    # Calculate averages
    mean_val_acc = np.mean(results_val_acc)
    mean_time_fit = np.mean(results_time_fit)
    mean_time_predict = np.mean(results_time_predict)

    # Calculate standard deviations
    std_val_acc = np.std(results_val_acc)

    metrix = {
        "accuracy_val_moy": mean_val_acc,
        "accuracy_val_std": std_val_acc,
        "time_fit_moy": mean_time_fit,
        "time_predict_moy": mean_time_predict
    }
    for run in range(nb_runs):
        metrix[f'accuracy_val_run_{run}'] = results_val_acc[run]

    # multiple metrics
    mlflow.log_metrics(metrix)

    results = {'model': model_name,
               'df': 'data_3_classes',
               'feature': feature,
               'accuracy_val_moy': mean_val_acc,
               'accuracy_val_std': std_val_acc,
               'time_fit_moy (s)': mean_time_fit,
               'time_predict_moy (s)': mean_time_predict,
              }

    # Append a new row for this model
    model_results.append(results)

    print('\n', f'Accuracy val {feature}, moyenne : {mean_val_acc}')
    print(f'Accuracy val {feature}, écart-type : {std_val_acc}')

    return metrix


# _ = test_optuna_n_times()


In [ ]:
with mlflow.start_run(run_name="optuna_1", experiment_id=experiment_id) as parent:
    parameters = {
        "model": "V4_optuna",
        "preprocessing": 'norm',
        "feature": 'originale, photo_path',
        'target': f'{nb_classes}_classes',
        'dim images': f'{size_wh} x {size_wh} x 3',
        'size val test sets': test_size,
        'nb epochs': epochs
    }
    mlflow.log_params(parameters)

    all_metrics = {}

    metrix = test_optuna_n_times()

    all_metrics[f'accuracy_val_moy_{feature}'] = metrix['accuracy_val_moy']

    print('\n')
    pprint(all_metrics)
    mlflow.log_metrics(all_metrics)


In [ ]:
# {'conv_1_units': 28, 'filter1': 3, 'conv_2_units': 4, 'filter2': 5, 'fc_1_units': 28}. Best is trial 8 with value: 0.6451612710952759.
# value: 0.6666666865348816 and parameters: {'conv_1_units': 8, 'filter1': 5, 'conv_2_units': 12, 'filter2': 5, 'fc_1_units': 80}

affichage_results(best_acc=best_acc)


### optimizer


In [ ]:
experiment_id = create_mlflow_experiment(
    experiment_name='optuna_2_optimizer',
    artifact_location="./mlruns/artifacts",
    tags={"model": "V4", "preprocessing": 'norm', "feature": "photo_path", 'target': '3_classes'},
)

experiment = get_mlflow_experiment(experiment_id=experiment_id)
print("Name: {}".format(experiment.name))


In [ ]:
#

def build_model(conv_1_units, filter1, conv_2_units, filter2, fc_1_units, optimizer):
    model = Sequential()
    model.add(Conv2D(conv_1_units, kernel_size=(filter1, filter1),
                     activation="tanh",
                     input_shape=(size_wh, size_wh, 3)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(conv_2_units, kernel_size=(filter2, filter2),
                     activation="tanh"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(fc_1_units, activation='tanh'))
    model.add(Dense(nb_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model


def objective(trial):
    # Define a pruning callback
    pruning_callback = TFKerasPruningCallback(trial, "val_accuracy")

    # Parameters to optimize
    conv_1_units = trial.suggest_int('conv_1_units', 4, 32, step=4)
    filter1 = trial.suggest_int('filter1', 3, 5, step=2)
    conv_2_units = trial.suggest_int('conv_2_units', 4, 32, step=4)
    filter2 = trial.suggest_int('filter2', 3, 5, step=2)
    fc_1_units = trial.suggest_int('fc_1_units', 10, 100, step=10)
    optimizer = trial.suggest_categorical('optimizer', ["Adam", "RMSprop", "SGD"])

    # Build and compile the model with the suggested parameters
    model = build_model(conv_1_units, filter1, conv_2_units, filter2, fc_1_units, optimizer)

    # Train the model
    history = model.fit(X_train, y_train_ohe, epochs=epochs, batch_size=32,
                        validation_data=(X_val, y_val_ohe),
                        callbacks=[pruning_callback], verbose=0)

    # Evaluate the model on the validation set
    val_acc = history.history['val_accuracy'][-1]

    return val_acc


In [ ]:
with mlflow.start_run(run_name="optuna_2", experiment_id=experiment_id) as parent:
    parameters = {
        "model": "V5_optuna",
        "preprocessing": 'norm',
        "feature": 'originale, photo_path',
        'target': f'{nb_classes}_classes',
        'dim images': f'{size_wh} x {size_wh} x 3',
        'size val test sets': test_size,
        'nb epochs': epochs
    }
    mlflow.log_params(parameters)

    all_metrics = {}

    metrix = test_optuna_n_times(model_name='V5_optuna')

    all_metrics[f'accuracy_val_moy_{feature}'] = metrix['accuracy_val_moy']

    print('\n')
    pprint(all_metrics)
    mlflow.log_metrics(all_metrics)


In [ ]:
# 0.6236559152603149 and parameters: {'conv_1_units': 20, 'filter1': 3, 'conv_2_units': 4, 'filter2': 5, 'fc_1_units': 10, 'optimizer': 'Adam'}
# 0.6129032373428345 and parameters: {'conv_1_units': 8, 'filter1': 3, 'conv_2_units': 8, 'filter2': 5, 'fc_1_units': 20, 'optimizer': 'Adam'}. Best is trial 7 with value: 0.6129032373428345.

affichage_results(best_acc=best_acc)


In [ ]:
# gridsize smaller grids ?

# Optimizer: The Adam optimizer is a good choice, but you might also experiment with other optimizers
# such as RMSprop or SGD (Stochastic Gradient Descent) with momentum.

# Loss Function: Cross-entropy loss is appropriate for classification tasks with softmax activation
# in the output layer, so categorical_crossentropy is fine.

# Model Capacity: LeNet is a relatively shallow network compared to modern architectures.
# Depending on the complexity of your dataset, you might need to adjust the model's capacity
# by adding more convolutional layers or increasing the number of units in the fully connected layers.

# Regularization: You may consider adding regularization techniques such as dropout or weight decay
# to prevent overfitting, especially if you observe overfitting during training.

# modify input shape ?

# inception module + global average pooling ?
